In [11]:
import pandas as pd
import numpy as np
import os
import pyarrow as pa
import pyarrow.parquet as pq

### 因子构造样例

#### 预先读取merge_daily_info

In [12]:
merge_daily_info = pd.read_parquet(r'/Users/xuyanye/Desktop/quant mm/factor/data/merge_daily_info.parquet')
merge_daily_info['TradingDate'] = pd.to_datetime(merge_daily_info['TradingDate'])
merge_daily_info = merge_daily_info.drop_duplicates(subset=['Stkcd','TradingDate'])
TRD_Dalyr = pd.read_parquet(r'/Users/xuyanye/Desktop/quant mm/factor/data/TRD_Dalyr.parquet')

In [20]:
TRD_Dalyr.columns

Index(['Stkcd', 'TradingDate', 'Opnprc', 'Hiprc', 'Loprc', 'Clsprc',
       'Dnshrtrd', 'Dnvaltrd', 'Dsmvosd', 'Dsmvtll', 'Dretwd', 'Dretnd',
       'Adjprcwd', 'Adjprcnd', 'Markettype', 'Capchgdt', 'Trdsta',
       'Ahshrtrd_D', 'Ahvaltrd_D', 'PreClosePrice', 'ChangeRatio', 'LimitDown',
       'LimitUp', 'LimitStatus'],
      dtype='object')

In [13]:
#数据提取与合并
Dnshrtrd = TRD_Dalyr[['TradingDate','Stkcd','Dnshrtrd']]
Dsmvosd = TRD_Dalyr[['TradingDate','Stkcd','Dsmvosd']]
Clsprc = TRD_Dalyr[['TradingDate','Stkcd','Clsprc']]
calc_cvturn = pd.merge(Dnshrtrd,
                             pd.merge(Dsmvosd, Clsprc, on=['TradingDate', 'Stkcd'], how='left'),
                             on=['TradingDate', 'Stkcd'], how='left')

In [8]:
calc_cvturn.columns

Index(['TradingDate', 'Stkcd', 'Dnshrtrd', 'Dsmvosd', 'Clsprc'], dtype='object')

# alpha019

In [15]:
COLUMN_FACTOR_NAME = "alpha019" 
ALPHA_SAVE_PATH_DIR = "/Users/xuyanye/Desktop/quant mm/factor/alpha"
if not os.path.exists(ALPHA_SAVE_PATH_DIR):
    os.makedirs(ALPHA_SAVE_PATH_DIR)
# 定义最终因子在DataFrame中的列名，以及Parquet文件的基础名称
# 这两部分必须一致，才能被您的回测代码正确识别

In [16]:
# --- 因子计算步骤 ---
# 假设 calc_cvturn 在此之前已经正确地准备好，
# 并且包含了 'TradingDate', 'Stkcd', 'Dnshrtrd', 'Dsmvosd', 'Clsprc' 列。
# 因子计算逻辑仅会使用 'Clsprc'。

calc_alpha019 = calc_cvturn.copy()

# 1. 确保数据按股票代码和交易日期排序
# 这一步对于计算时间序列相关的函数（如 diff, rolling）至关重要
calc_alpha019 = calc_alpha019.sort_values(by=['Stkcd', 'TradingDate']).copy()

# 2. 计算 delta(close, 7) - 对应 Clsprc 7个交易日的日变化
# 公式中的 (close - delay(close, 7)) + delta(close, 7)
# 实际上等价于 2 * (close - delay(close, 7)) 或 2 * delta(close, 7)
calc_alpha019['delta_close_7'] = calc_alpha019.groupby('Stkcd')['Clsprc'].diff(7)

# 3. 计算 sign(((close - delay(close, 7)) + delta(close, 7)))
# 也就是 sign(2 * delta(close, 7))
calc_alpha019['sign_term'] = np.sign(2 * calc_alpha019['delta_close_7'])

# 4. 计算 daily returns (日收益率)
calc_alpha019['daily_returns'] = calc_alpha019.groupby('Stkcd')['Clsprc'].pct_change()

# 5. 计算 sum(returns, 250)
# rolling(window=250, min_periods=1) 表示计算250个周期的和，
# min_periods=1 允许在数据不足250个点时也进行计算（例如每个股票的前250个交易日）
calc_alpha019['sum_returns_250'] = calc_alpha019.groupby('Stkcd')['daily_returns'].rolling(window=250, min_periods=1).sum().reset_index(level=0, drop=True)

# 6. 计算 (1 + sum(returns, 250))
calc_alpha019['one_plus_sum_returns'] = 1 + calc_alpha019['sum_returns_250']

# 7. 计算 rank((1 + sum(returns, 250)))
# 排名通常是按TradingDate进行跨截面排名
calc_alpha019['ranked_term'] = calc_alpha019.groupby('TradingDate')['one_plus_sum_returns'].rank(method='average', ascending=True)

# 8. 计算 (1 + rank((1 + sum(returns, 250))))
calc_alpha019['one_plus_ranked_term'] = 1 + calc_alpha019['ranked_term']

# 9. 计算最终的 Alpha019 因子值
# Alpha019 = (-1 * sign_term) * (one_plus_ranked_term)
calc_alpha019['alpha019'] = -1 * calc_alpha019['sign_term'] * calc_alpha019['one_plus_ranked_term']

    Stkcd TradingDate  alpha019
7       1  1991-04-12       6.0
8       1  1991-04-13       2.0
9       1  1991-04-16       6.0
10      1  1991-04-17       6.0
11      1  1991-04-18       4.0


In [19]:
# 假设 merge_daily_info 是另一个包含 Stkcd 和 TradingDate 的 DataFrame
# 将计算出的 Alpha12_Factor 合并到 merge_daily_info
# 注意：如果 calc_cvturn 就是最终要用的 DataFrame，可能不需要这一步合并
# 这里为了与用户提供的格式保持一致，假设有一个目标 DataFrame 'merge_daily_info'
# 如果 calc_cvturn 本身就是最终结果，则可以跳过此合并步骤
# 请根据实际情况调整 'merge_daily_info' 的定义
try:
    # 模拟 merge_daily_info 如果它不存在
    if 'merge_daily_info' not in locals():
        print("警告：'merge_daily_info' 未定义。为演示目的，将假设 merge_daily_info 等同于 calc_cvturn 的基础部分。")
        merge_daily_info = calc_cvturn[['Stkcd', 'TradingDate']].copy() # 只是一个示例

    # 进行合并
    calc_cvturn = pd.merge(merge_daily_info,
                           calc_alpha019[['Stkcd','TradingDate','alpha019']],
                           on=['Stkcd','TradingDate'],
                           how='left')
except NameError:
    print("错误：'merge_daily_info' 未定义，请确保该DataFrame存在后再执行合并。")


# 去掉缺失值与inf (根据用户提供的格式)
# 这一步会移除 Alpha12_Factor 列中因为 delta 计算导致的 NaN 值 (例如每个股票的第一个交易日)
calc_cvturn = calc_cvturn.dropna(subset=['alpha019']) # 只针对因子列去除NaN，避免影响其他数据
calc_cvturn = calc_cvturn[~np.isinf(calc_cvturn['alpha019'])] # 确保因子值不是无穷大

# 打印结果 DataFrame 的头部，以供检查
print(calc_cvturn.head())

    Stkcd TradingDate  alpha019
7       1  1991-04-12       6.0
8       1  1991-04-13       2.0
9       1  1991-04-16       6.0
10      1  1991-04-17       6.0
11      1  1991-04-18       4.0


In [21]:
##采用parquet优化内存
save_path = r'/Users/xuyanye/Desktop/quant mm/factor/alpha/alpha019.parquet'
calc_cvturn.to_parquet(save_path, compression='gzip')